In [1]:
!git clone https://github.com/valkryhx/chatGLM-6B-QLoRA

fatal: destination path 'chatGLM-6B-QLoRA' already exists and is not an empty directory.


In [2]:
%cd /kaggle/working/chatGLM-6B-QLoRA
!git pull --all --force
!ls
!pip install -r requirements.txt

/kaggle/working/chatGLM-6B-QLoRA
Fetching origin
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 14 (delta 7), reused 1 (delta 0), pack-reused 0
Unpacking objects: 100% (14/14), 28.92 KiB | 3.61 MiB/s, done.
From https://github.com/valkryhx/chatGLM-6B-QLoRA
   2e0e6b9..16655be  main       -> origin/main
Updating 2e0e6b9..16655be
Fast-forward
 data/sft/add_prompt_anli_1.json | 21 ++++++++++++++
 data/sft/add_prompt_anli_2.json | 63 +++++++++++++++++++++++++++++++++++++++++
 data/sft/anli0728.json          | 63 -----------------------------------------
 data/sft/anli_0727.json         | 21 --------------
 4 files changed, 84 insertions(+), 84 deletions(-)
 create mode 100644 data/sft/add_prompt_anli_1.json
 create mode 100644 data/sft/add_prompt_anli_2.json
 delete mode 100644 data/sft/anli0728.json
 delete mode 100644 data/sft/anli_0727.json
README.md		    pics
chatGLM_6B_QLoRA.json	    p

# 注意 data目录中不要有空白内容的txt 否则会出现长度为4左右的短sample 影响训练dataset的质量
# <font color=orange>虽然不报错 但是会让total length出现=4的奇怪现象 排查了很久才找到原因</font>

In [ ]:
!git pull --all --force 
#!pip install  -U git+https://github.com/huggingface/peft.git   # 20230717 peft==0.4.0正式发布了 不用调版本了推理完后再训练需要重新升级到0.4.0dev 所以有这句
!deepspeed --include localhost:0,1  pretrain_qlora_chatglm2.py \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-pt-anli-21 \
  --num_train_samples -1 \
  --num_eval_samples 12 \
  --block_size 512 \
  --train_data_path ./data/pretrain/anli_21 \
  --eval_data_path  ./data/pretrain/anli_21 \
  --max_length 1512 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 1  \
  --gradient_accumulation_steps 1 \
  --learning_rate  1e-5 \
  --num_train_epochs  50  \
  --save_total_limit 2 \
  --load_in_4bit True \
--deepspeed ds_zero2_config.json

# 从上一次的某个ckpt中训练
# --resume_from_checkpoint   ./output-pt-anli-21

In [ ]:
!git pull --all --force 
#!pip install  -U git+https://github.com/huggingface/peft.git   # 20230717 peft==0.4.0正式发布了 不用调版本了推理完后再训练需要重新升级到0.4.0dev 所以有这句
!deepspeed --include localhost:0,1  pretrain_qlora_chatglm2.py \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-pt-anli-21-v2 \
  --num_train_samples -1 \
  --num_eval_samples 12 \
  --block_size 512 \
  --train_data_path ./data/pretrain/anli_21 \
  --eval_data_path  ./data/pretrain/anli_21 \
  --resume_from_checkpoint ./output-pt-anli-21 \
  --max_length 512 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 1  \
  --gradient_accumulation_steps 1 \
  --learning_rate  1e-5 \
  --num_train_epochs  50  \
  --save_total_limit 2 \
  --load_in_4bit True \
--deepspeed ds_zero2_config.json

# 推理

In [2]:
import os
import argparse
from typing import List, Dict, Optional
from accelerate import init_empty_weights  # load an empty model,just structure , no real weight.
import bitsandbytes as bnb
import torch
from glob import glob
from loguru import logger
from datasets import load_dataset
from transformers import (
    AutoModel,
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    set_seed,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig 
)
from peft import (
    TaskType,
    LoraConfig,
    #AdaLoraConfig ,  #  提出自2020年 感觉和lora区别不大 而且和qlora有冲突 这里代码没有用到 
                     #例子https://www.zhihu.com/question/596950521/answer/3109759716
    get_peft_model,
    set_peft_model_state_dict,
    prepare_model_for_kbit_training
)
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING
from transformers.deepspeed import HfDeepSpeedConfig
import deepspeed
import json
from itertools import chain

[2023-08-01 08:15:59,949] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/lib/x86_64-linux-gnu'), PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/cuda/lib')}
  warn(msg)


# 预训练之后的模型推理

In [3]:
!git pull --all --force
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer , AutoModel
import torch
#from testglm.modeling_chatglm import ChatGLMForConditionalGeneration
device = 'cuda'


peft_model_path= "/kaggle/working/chatGLM-6B-QLoRA/output-pt-anli-21-v2/checkpoint-240"
config = PeftConfig.from_pretrained(peft_model_path)
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=torch.float16)
# base_model加载时保持和train时完全一致的参数配置
model = AutoModel.from_pretrained(config.base_model_name_or_path,
                                       quantization_config=q_config,
                                       trust_remote_code=True,
                                       device_map='auto')


Fetching origin
Already up to date.


A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm2-6b:
- configuration_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm2-6b:
- quantization.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm2-6b:
- modeling_chatglm.py
- quantization.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:

model = PeftModel.from_pretrained(model, peft_model_path)

#下面这三行貌似可以不用
#peft_model_dict = model.state_dict()
#peft_model_dict.update(torch.load(f"{peft_model_path}/adapter_model.bin"))
#model.load_state_dict(peft_model_dict,strict=False)

model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): ChatGLMForConditionalGeneration(
      (transformer): ChatGLMModel(
        (embedding): Embedding(
          (word_embeddings): Embedding(65024, 4096)
        )
        (rotary_pos_emb): RotaryEmbedding()
        (encoder): GLMTransformer(
          (layers): ModuleList(
            (0-27): 28 x GLMBlock(
              (input_layernorm): RMSNorm()
              (self_attention): SelfAttention(
                (query_key_value): Linear4bit(
                  in_features=4096, out_features=4608, bias=True
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=64, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=64, out_features=4608, bias=False)
                  )
          

# 查看lora_A/B 的weight 都很小

In [6]:
print(model.base_model.model.transformer.encoder.layers[27].self_attention.query_key_value.lora_A.default.weight)

Parameter containing:
tensor([[-0.0133,  0.0089,  0.0108,  ..., -0.0047,  0.0137, -0.0019],
        [ 0.0080, -0.0068,  0.0153,  ..., -0.0142, -0.0125,  0.0088],
        [-0.0077, -0.0129,  0.0101,  ..., -0.0008,  0.0148,  0.0007],
        ...,
        [-0.0133, -0.0021, -0.0146,  ..., -0.0012, -0.0005, -0.0090],
        [-0.0143,  0.0025, -0.0066,  ...,  0.0107, -0.0009, -0.0018],
        [-0.0053, -0.0098, -0.0012,  ..., -0.0121,  0.0130, -0.0031]],
       device='cuda:1')


In [7]:
print(model.base_model.model.transformer.encoder.layers[27].self_attention.query_key_value.lora_B.default.weight)

Parameter containing:
tensor([[ 0.0065, -0.0032, -0.0008,  ..., -0.0034,  0.0047,  0.0074],
        [-0.0029,  0.0017,  0.0011,  ...,  0.0027,  0.0011, -0.0025],
        [ 0.0038, -0.0003, -0.0030,  ..., -0.0031,  0.0008,  0.0028],
        ...,
        [-0.0017,  0.0023,  0.0003,  ...,  0.0010, -0.0022, -0.0010],
        [ 0.0019, -0.0005, -0.0002,  ...,  0.0017,  0.0017,  0.0010],
        [ 0.0022,  0.0003, -0.0032,  ..., -0.0021,  0.0004,  0.0022]],
       device='cuda:1')


# qkv module

In [8]:
print(model.base_model.model.transformer.encoder.layers[27].self_attention.query_key_value.weight)

Parameter containing:
Parameter(Params4bit([[115],
            [214],
            [ 49],
            ...,
            [100],
            [152],
            [ 85]], device='cuda:1', dtype=torch.uint8))


# dense module

In [12]:
print(model.base_model.model.transformer.encoder.layers[27].self_attention.dense.weight)

Parameter containing:
Parameter(Params4bit([[ 87],
            [150],
            [180],
            ...,
            [150],
            [ 58],
            [ 89]], device='cuda:1', dtype=torch.uint8))


# core attention 是个dropout层 没有参数正常

In [13]:
print(model.base_model.model.transformer.encoder.layers[27].self_attention.core_attention)

CoreAttention(
  (attention_dropout): Dropout(p=0.0, inplace=False)
)


In [31]:
torch.cuda.empty_cache()

In [ ]:
# 在二次预训练的基础上再sft 试试

In [5]:
#cat /kaggle/working/chatGLM-6B-QLoRA/data/sft/add_prompt_anli_2.json

# <font color=red>SFT的数据格式一定要注意和chatglm2的格式保持一致  不然微调和si'hu</font>
#   [Round {可以填1}]\n\n问：{}\n\n答：
# https://huggingface.co/THUDM/chatglm2-6b/blob/main/tokenization_chatglm.py#L168

In [6]:
!git pull --all --force 
#!pip install  -U git+https://github.com/huggingface/peft.git   # 20230717 peft==0.4.0正式发布了 不用调版本了推理完后再训练需要重新升级到0.4.0dev 所以有这句
!deepspeed --num_gpus=2  train_qlora_deepspeed_zero.py \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-sft-anli_0730 \
  --resume_from_checkpoint ./output-pt-anli-21-v2 \
  --num_eval_samples 50 \
  --train_data_path ./data/sft \
  --eval_data_path  ./data/sft \
  --max_input_length 256 \
  --max_output_length 1000 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 1 \
  --per_device_eval_batch_size 1  \
  --gradient_accumulation_steps 1 \
  --learning_rate  5e-5 \
  --num_train_epochs  40  \
  --save_total_limit 2 \
  --load_in_4bit True \
--deepspeed ds_zero2_config.json

Fetching origin
Already up to date.
[2023-08-01 12:02:20,429] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}

In [7]:
import os
import argparse
from typing import List, Dict, Optional
from accelerate import init_empty_weights  # load an empty model,just structure , no real weight.
import bitsandbytes as bnb
import torch
from glob import glob
from loguru import logger
from datasets import load_dataset
from transformers import (
    AutoModel,
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    set_seed,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig 
)
from peft import (
    TaskType,
    LoraConfig,
    #AdaLoraConfig ,  #  提出自2020年 感觉和lora区别不大 而且和qlora有冲突 这里代码没有用到 
                     #例子https://www.zhihu.com/question/596950521/answer/3109759716
    get_peft_model,
    set_peft_model_state_dict,
    prepare_model_for_kbit_training
)
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING
from transformers.deepspeed import HfDeepSpeedConfig
import deepspeed
import json
from itertools import chain

[2023-08-01 12:38:30,928] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /opt/conda/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/lib/x86_64-linux-gnu'), PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/cuda/lib')}
  warn(msg)


In [8]:
# PT + SFT
!git pull --all --force
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer , AutoModel
import torch
#from testglm.modeling_chatglm import ChatGLMForConditionalGeneration
device = 'cuda'


peft_model_path= "/kaggle/working/chatGLM-6B-QLoRA/output-sft-anli_0730/checkpoint-390"
config = PeftConfig.from_pretrained(peft_model_path)
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=torch.float16)
# base_model加载时保持和train时完全一致的参数配置
model = AutoModel.from_pretrained(config.base_model_name_or_path,
                                       quantization_config=q_config,
                                       trust_remote_code=True,
                                       device_map='auto')

Fetching origin
Already up to date.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

# PT SFT之前的原始基座模型 推理

In [10]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)

print("用户：你好，智能小助手\n")
old_response = ""
for response, history in model.stream_chat(tokenizer, "你好", [], max_length=256, 
                                           top_p=0.7, temperature=0.95,
                                           num_beams=2):
    print(response[len(old_response):], end="")
    old_response = response
print(end="\r")
#print(1111,old_response)作为烽火智能助手，我遇到了以下故障现象，某地市某网络单盘占用槽位显示异常现场实际占用两个槽位。请给出类似案例以供参考。
#print("ChatGLM2-6B：\n",response)
#print("\n------------------------------------------------\n用户：")
#raise ValueError(123)
line = input()
while line:
    if line.strip()=="#QUIT": # 输入#QUIT退出对话循环
        break
    #inst = "我有关于烽火SSE的问题，请你作为烽火SSE智能助手，帮我回答下面的问题。" #计算两个矩阵的乘积："
    #line = PROMPT_TEMPLATE.format_map({'instruction': line})
    if line.strip()=="#CLEAR" :  #清空历史对话
        history=[]
    #line = inst  + line
    #print(f"【formatted line】={line}")
    #old_response = ""
    print("num_beams=10")
    for response, history in model.stream_chat(tokenizer=tokenizer, 
                                               history=[],#history ,
                                               max_length=1024,
                                               query=line,
                                               top_p=0.95,temperature=0.45,
                                               num_beams=10):
        print(response[len(old_response):], end="")
        old_response = response
    print(end="\r")
    #print(f"\nold response=\n{old_response}")
    print(end="\r") # 量化后 num_beams只能=1 否则报错
    print("ChatGLM2-6B：\n", old_response.replace("\\n",'\n'))
    print("\n------------------------------------------------\n用户：")
    line = input()

用户：你好，智能小助手



 作为烽火智能助手，我遇到了以下故障现象，某工程上，集团OTMS采集发现我司OCP单盘TRXB-2/OCH-1、TRXB-1/OCH-1历史性能没有，当前性能有上报。请给出类似案例以供参考。


num_beams=10
烽火智能助手，我遇到了以下故障现象，某工程上，集团OTMS采集发现我司OCP单盘TRXB-2/OCH-1、TRXB-1/OCH-1历史性能没有，当前性能有上报。请给出类似案例以供参考。

类似案例：

1. 某公司网络设备，OTMS采集发现历史性能数据没有，但当前性能数据有上报。
2. 某公司数据中心，OTMS采集发现某个服务器的网络性能数据没有，但当前性能数据有上报。
3. 某公司云服务器，OTMS采集发现某个云服务的网络性能数据没有，但当前性能数据有上报。
4. 某公司物联网设备，OTMS采集发现某个设备的网络性能数据没有，但当前性能数据有上报。

ChatGLM2-6B：发现某个网络设备的性能数据没有，但当前性能数据有上报。可能是由于设备上的缓存数据没有清除干净或者设备上的数据没有更新导致的。建议先检查设备上的缓存数据，并确保设备上的数据是最新更新的。如果问题仍然存在，请检查设备配置和网络连接，并确保设备能够正常地与其他设备通信。
 作为烽火智能助手，我遇到了以下故障现象，某工程上，集团OTMS采集发现我司OCP单盘TRXB-2/OCH-1、TRXB-1/OCH-1历史性能没有，当前性能有上报。请给出类似案例以供参考。

类似案例：

1. 某公司网络设备，OTMS采集发现历史性能数据没有，但当前性能数据有上报。
2. 某公司数据中心，OTMS采集发现某个服务器的网络性能数据没有，但当前性能数据有上报。
3. 某公司云服务器，OTMS采集发现某个云服务的网络性能数据没有，但当前性能数据有上报。
4. 某公司物联网设备，OTMS采集发现某个设备的网络性能数据没有，但当前性能数据有上报。

这些案例都是OTMS采集发现某个网络设备的性能数据没有，但当前性能数据有上报。可能是由于设备上的缓存数据没有清除干净或者设备上的数据没有更新导致的。建议先检查设备上的缓存数据，并确保设备上的数据是最新更新的。如果问题仍然存在，请检查设备配置和网络连接，并确保设备能够正常地与其他设备通信。

------------------------------------------------
用户：


 作为烽火智能助手，我遇到了以下故障现象，现场将网管从备用切回主用，并且主用服务器上网管补丁从UMC2-T V1R3SPH0002.16加载到UMC2-T V1R3SPH0002.20后，启动服务发现DHCPserver和onlinehelpserver服务为去使能状态且无法手动拉起来。请给出类似案例以供参考。


num_beams=10
烽火智能助手，我遇到了以下故障现象：现场将网管从备用切回主用，并且主用服务器上网管补丁从UMC2-T V1R3SPH0002.16加载到UMC2-T V1R3SPH0002.20后，启动服务发现DHCPserver和onlinehelpserver服务为去使能状态且无法手动拉起来。

类似案例：

1. 现场将网管从主用切回备用，并且主用服务器上网管补丁从UMC2-T V1R3SPH0002.16加载到UMC2-T V1R3SPH0002.20后，启动服务发现DHCPserver和onlinehelpserver服务为去使能状态且无法手动拉起来。

2. 现场将网管从主用切回备用，并且主用服务器上网管补丁从UMC2-T V1R3SPH0002.16加载到UMC2-T V1R3SPH0002.20后，启动服务发现DHCPserver和onlinehelpserver服务为去使能状态且无法手动拉起来。

3. 现场将网管从主用切回备用，并且主用服务器上网管补丁从UMC2-T V1R3SPH0002.16加载到UMC2-T V1R3SPH0002.20后，启动服务发现DHCPserver和onlinehelpserver服务为去使能状态且无法手动拉起来。

4. 现场将网管从主用切回备用，并且主用服务器上网管补丁从UMC2-T V1R3SPH0002.16加载到UMC2-T V1R3SPH0002.20后，启动服务发现DHCPserver和onlinehelpserver服务为去使能状态且无法手动拉起来。

5. 现场将网管从主用切回备用，并且主用服务器上网管补丁从UMC2-T V1R3SPH0002.16加载到UMC2-T V1R3SPH0002.20后，启动服务发现DHCPserver和onlinehelpserver服务为去使能状态且无法手动拉起来。

ChatGLM2-6B：题类似，可以作为参考。
 作为烽火智能助手，我遇到了以下故障现象：现场将网管从备用切回主用，并且主用服务器上网管补丁从UMC2-T V1R3SPH0002.16加载到UMC2-T V1R3SPH0002.20后，启动服务发现DHCPserver和onlinehelpserver服务为去使能状态且无法手动拉起来。

类似案例：

1. 现场将网管从主用切回备用，并且主用

 作为烽火智能助手，我遇到了以下故障现象，某工程网管版本升级到UMC2-T V1R3版本，登录网管FTP账号密码不对，性能无法查询。请给出类似案例以供参考。


num_beams=10
烽火智能助手，我遇到了以下故障现象，某工程网管版本升级到UMC2-T V1R3版本，登录网管FTP账号密码不对，性能无法查询。请给出类似案例以供参考。

类似案例：

1. 某企业网管系统版本更新后，用户登录账号密码出现错误，无法登录系统。

2. 某高校实验室的实验系统在更新后，部分用户无法登录系统，提示账号密码错误。

3. 某金融机构的客户管理系统在版本更新后，部分用户无法登录系统，提示账号密码错误。

4. 某医疗机构的HIS系统在更新后，部分医生无法登录系统，提示账号密码错误。

5. 某工厂的生产管理系统在更新后，部分员工无法登录系统，提示账号密码错误。

针对以上类似案例，可以尝试以下步骤进行解决：

1. 对于版本更新导致的登录问题，可以尝试使用备份的账号密码进行登录，或者联系管理员进行重置。

2. 对于密码错误导致的登录问题，可以尝试使用“忘记密码”功能进行重置，或者联系管理员进行重置。

3. 对于性能无法查询的问题，可以尝试使用其他工具进行查询，或者联系管理员进行查询。

ChatGLM2-6B：问题，可以尝试查看系统文档或者联系技术支持进行解决。
 作为烽火智能助手，我遇到了以下故障现象，某工程网管版本升级到UMC2-T V1R3版本，登录网管FTP账号密码不对，性能无法查询。请给出类似案例以供参考。

类似案例：

1. 某企业网管系统版本更新后，用户登录账号密码出现错误，无法登录系统。

2. 某高校实验室的实验系统在更新后，部分用户无法登录系统，提示账号密码错误。

3. 某金融机构的客户管理系统在版本更新后，部分用户无法登录系统，提示账号密码错误。

4. 某医疗机构的HIS系统在更新后，部分医生无法登录系统，提示账号密码错误。

5. 某工厂的生产管理系统在更新后，部分员工无法登录系统，提示账号密码错误。

针对以上类似案例，可以尝试以下步骤进行解决：

1. 对于版本更新导致的登录问题，可以尝试使用备份的账号密码进行登录，或者联系管理员进行重置。

2. 对于密码错误导致的登录问题，可以尝试使用“忘记密码”功能进行重置，或者联系管理员进行重置。

3. 对于性能无法查询的问题，可以尝试使用其他工具进行查询，或者联系管理员进行查询。

4. 对于其他类型的故障问题，可以尝试查看系统文档或者联系技术支持进行

 #QUIT


# PT+SFT 之后的推理

In [11]:
model = PeftModel.from_pretrained(model, peft_model_path)

#下面这三行貌似可以不用
#peft_model_dict = model.state_dict()
#peft_model_dict.update(torch.load(f"{peft_model_path}/adapter_model.bin"))
#model.load_state_dict(peft_model_dict,strict=False)

model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): ChatGLMForConditionalGeneration(
      (transformer): ChatGLMModel(
        (embedding): Embedding(
          (word_embeddings): Embedding(65024, 4096)
        )
        (rotary_pos_emb): RotaryEmbedding()
        (encoder): GLMTransformer(
          (layers): ModuleList(
            (0-27): 28 x GLMBlock(
              (input_layernorm): RMSNorm()
              (self_attention): SelfAttention(
                (query_key_value): Linear4bit(
                  in_features=4096, out_features=4608, bias=True
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=64, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=64, out_features=4608, bias=False)
                  )
          

In [12]:
print(model.base_model.model.transformer.encoder.layers[27].self_attention.query_key_value.lora_A.default.weight)



Parameter containing:
tensor([[-0.0101,  0.0081,  0.0151,  ..., -0.0049,  0.0067, -0.0053],
        [ 0.0059, -0.0041,  0.0189,  ..., -0.0143, -0.0016,  0.0011],
        [-0.0104, -0.0077,  0.0076,  ..., -0.0080,  0.0119,  0.0015],
        ...,
        [-0.0176, -0.0040, -0.0201,  ..., -0.0007,  0.0075, -0.0084],
        [-0.0030,  0.0050,  0.0006,  ...,  0.0124, -0.0048, -0.0102],
        [ 0.0023, -0.0137,  0.0019,  ..., -0.0082,  0.0100, -0.0034]],
       device='cuda:1')


In [13]:
print(model.base_model.model.transformer.encoder.layers[27].self_attention.query_key_value.lora_B.default.weight)

Parameter containing:
tensor([[ 7.3729e-03, -6.7130e-03,  1.0771e-02,  ..., -6.3787e-03,
          3.7178e-02,  2.3179e-03],
        [-6.1272e-03,  6.7499e-03, -7.5754e-04,  ...,  1.1679e-02,
          7.8110e-03, -7.0474e-03],
        [ 1.5274e-02, -2.8465e-03, -2.5195e-02,  ..., -1.7403e-02,
         -2.6408e-02,  1.5521e-02],
        ...,
        [-6.3531e-03,  2.9100e-03,  4.3314e-03,  ..., -9.7776e-04,
         -4.4417e-04,  6.6712e-05],
        [-1.8721e-03, -8.8557e-04,  1.4184e-03,  ...,  5.4086e-03,
         -5.6311e-03, -2.0789e-03],
        [ 1.3073e-03,  8.9136e-03, -7.6425e-03,  ..., -6.9006e-03,
         -3.5105e-03, -6.0867e-04]], device='cuda:1')


In [14]:
print(model.base_model.model.transformer.encoder.layers[27].self_attention.query_key_value.weight)

Parameter containing:
Parameter(Params4bit([[115],
            [214],
            [ 49],
            ...,
            [100],
            [152],
            [ 85]], device='cuda:1', dtype=torch.uint8))


# stream chat 

In [23]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)

print("用户：你好，智能小助手\n")
old_response = ""
for response, history in model.stream_chat(tokenizer, "你好", [], max_length=1024, 
                                           top_p=0.7, temperature=0.95,
                                           num_beams=5):
    print(response[len(old_response):], end="")
    old_response = response
print(end="\r")
#print(1111,old_response)作为烽火智能助手，我遇到了以下故障现象，某地市某网络单盘占用槽位显示异常现场实际占用两个槽位。请给出类似案例以供参考。
#print("ChatGLM2-6B：\n",response)
#print("\n------------------------------------------------\n用户：")
#raise ValueError(123)
line = input()
while line:
    if line.strip()=="#QUIT": # 输入#QUIT退出对话循环
        break
    #inst = "我有关于烽火SSE的问题，请你作为烽火SSE智能助手，帮我回答下面的问题。" #计算两个矩阵的乘积："
    #line = PROMPT_TEMPLATE.format_map({'instruction': line})
    if line.strip()=="#CLEAR" :  #清空历史对话
        history=[]
    #line = inst  + line
    #print(f"【formatted line】={line}")
    #old_response = ""
    beams =2
    print(f"num_beams={beams}")
    for response, history in model.stream_chat(tokenizer=tokenizer, 
                                               history=[],#history ,
                                               max_length=1024,
                                               query=line,
                                               top_p=0.95,temperature=0.55,
                                               num_beams=beams):
        print(response[len(old_response):], end="")
        old_response = response
    print(end="\r")
    #print(f"\nold response=\n{old_response}")
    print(end="\r") # 量化后 num_beams只能=1 否则报错
    print("ChatGLM2-6B：\n", old_response.replace("\\n",'\n'))
    print("\n------------------------------------------------\n用户：")
    line = input()

用户：你好，智能小助手



 案例1 {"instruction": "[Round 1]\n\n问：作为烽火智能助手，我遇到了以下故障现象，登录UNM2000网管，部分网元分别选择在网元侧和网管侧查询历史性能，得到的数据不同，网管侧读取为错误。部分网元无历史性能。。请给出类似案例以供参考。\n\n答：", "input": "", "output": "案例主题:集约化网管中一个地市钟部分网元历史性能错乱、部分网元无历史性能\n案例编号:AL202204093411\n网络场景:某地市OTN集约化网管，网管版本V4R1M11SPH0115。\n现象描述:登录UNM2000网管，部分网元分别选择在网元侧和网管侧查询历史性能，得到的数据不同，网管侧读取为错误。部分网元无历史性能。\n处理过程:使用SQLYOG软件登录PAS数据库，查看PAS库中网元的管理网块号网元号与网管界面中网元的管理网块号网元号对比，发现不一致，将网元属性重新保存后，触发pas增量同步，使pas库中网块网元号修正，问题解决。\n根因:PAS库中网块网元号与网管不一致，导致网管侧读取历史性能时，读取的是网管库中网块网元号的PAS历史性能数据，网元侧读取历史性能时，读取的是manager上报的设备中的性能数据。该问题是由配置融合中某些异常操作导致。\n解决方案:将网元属性重新保存后，触发pas增量同步，使pas库中网块网元号修正，问题解决。\n原因分类:配置和操作类问题\n"} 案例2 {"instruction": "[Round 1]\n\n问：作为烽火智能助手，我遇到了以下故障现象，某地市移动OTN网管，在客户资源池的虚拟机上部署了主备集群的集中式网管，某日发现：1、无法通过unm2000_client根目录底下的unm2000.exe文件打开网管客户端，不提示任何报错；2、可以通过D:\\unm2000_Client\\unm2000\\bin 目录下的unm2000.exe 文件打开网管客户端；3、IceGridGUI工具无法打开，不提示任何报错。。请给出类似案例以供参考。\n\n答：", "input": "", "output": "案例主题:由于虚拟内存太小导致客户端闪退，IceGridGui无法打开的技术案例\n案例编号:AL2023050512372\n网络场景:\n现象描述:某地市移动OTN网管，在客户资源

num_beams=2
ChatGLM2-6B：
 你好👋！我是人工智能助手 ChatGLM2-6B，很高兴见到你，欢迎问我任何问题。

------------------------------------------------
用户：


 案例1 {"instruction": "[Round 1]\n\n问：作为烽火智能助手，我遇到了以下故障现象，登录UNM2000网管，部分网元分别选择在网元侧和网管侧查询历史性能，得到的数据不同，网管侧读取为错误。部分网元无历史性能。。请给出类似案例以供参考。\n\n答：", "input": "", "output": "案例主题:集约化网管中一个地市钟部分网元历史性能错乱、部分网元无历史性能\n案例编号:AL202204093411\n网络场景:某地市OTN集约化网管，网管版本V4R1M11SPH0115。\n现象描述:登录UNM2000网管，部分网元分别选择在网元侧和网管侧查询历史性能，得到的数据不同，网管侧读取为错误。部分网元无历史性能。\n处理过程:使用SQLYOG软件登录PAS数据库，查看PAS库中网元的管理网块号网元号与网管界面中网元的管理网块号网元号对比，发现不一致，将网元属性重新保存后，触发pas增量同步，使pas库中网块网元号修正，问题解决。\n根因:PAS库中网块网元号与网管不一致，导致网管侧读取历史性能时，读取的是网管库中网块网元号的PAS历史性能数据，网元侧读取历史性能时，读取的是manager上报的设备中的性能数据。该问题是由配置融合中某些异常操作导致。\n解决方案:将网元属性重新保存后，触发pas增量同步，使pas库中网块网元号修正，问题解决。\n原因分类:配置和操作类问题\n"} 案例2 {"instruction": "[Round 1]\n\n问：作为烽火智能助手，我遇到了以下故障现象，某地市移动OTN网管，在客户资源池的虚拟机上部署了主备集群的集中式网管，某日发现：1、无法通过unm2000_client根目录底下的unm2000.exe文件打开网管客户端，不提示任何报错；2、可以通过D:\\unm2000_Client\\unm2000\\bin 目录下的unm2000.exe 文件打开网管客户端；3、IceGridGUI工具无法打开，不提示任何报错。。请给出类似案例以供参考。\n\n答：", "input": "", "output": "案例主题:由于虚拟内存太小导致客户端闪退，IceGridGui无法打开的技术案例\n案例编号:AL2023050512372\n网络场景:\n现象描述:某地市移动OTN网管，在客户资源

num_beams=2
ChatGLM2-6B：
 你好👋！我是人工智能助手 ChatGLM2-6B，很高兴见到你，欢迎问我任何问题。

------------------------------------------------
用户：


...

 登录UNM2000网管，部分网元分别选择在网元侧和网管侧查询历史性能，得到的数据不同，网管侧读取为错误。部分网元无历史性能。。请给出类似案例以供参考。


num_beams=2
主题:集约化网管中一个地市钟部分网元历史性能错乱、部分网元无历史性能
案例编号:AL20220928522311922
网络场景:某地市OTN集约化网管，网管侧读取历史性能，部分网元选择在网管侧查询，读取到错误数据，而部分网元则无历史性能。
现象描述:登录UNM2000网管，部分网元分别选择在网元侧和网管侧查询历史性能，得到的数据不同，网管侧读取为错误。部分网元无历史性能。
处理过程:使用SQLYOG软件登录PAS数据库，查看PAS库中网元的管理网块号网元号与网管界面中网元的管理网块号网元号对比，发现不一致，将网元属性重新保存后，触发pas增量同步，使pas库中网块网元号修正，问题解决。
根因:PAS库中网块网元号与网管不一致，导致网管侧读取历史性能时，读取到的是网管库中网块网元号的PAS历史性能数据，网元侧读取历史性能时，读取到的是manager上报的设备中的性能数据。该问题是由配置融合中某些异常操作导致。
解决方案:将网元属性重新保存后，触发pas增量同步，使pas库中网块网元号修正，问题解决。
ChatGLM2-6B：题
 案例主题:集约化网管中一个地市钟部分网元历史性能错乱、部分网元无历史性能
案例编号:AL20220928522311922
网络场景:某地市OTN集约化网管，网管侧读取历史性能，部分网元选择在网管侧查询，读取到错误数据，而部分网元则无历史性能。
现象描述:登录UNM2000网管，部分网元分别选择在网元侧和网管侧查询历史性能，得到的数据不同，网管侧读取为错误。部分网元无历史性能。
处理过程:使用SQLYOG软件登录PAS数据库，查看PAS库中网元的管理网块号网元号与网管界面中网元的管理网块号网元号对比，发现不一致，将网元属性重新保存后，触发pas增量同步，使pas库中网块网元号修正，问题解决。
根因:PAS库中网块网元号与网管不一致，导致网管侧读取历史性能时，读取到的是网管库中网块网元号的PAS历史性能数据，网元侧读取历史性能时，读取到的是manager上报的设备中的性能数据。该问题是由配置融合中某些异常操作导致。
解决方案:将网元属性重新保存后，触发pas增量同步，使pas库中网块网元号修正，问题解决。
原因分类:配置和操作类问题

------------------------------------------

 请用7种不同的方式表达这句话：登录UNM2000网管，部分网元分别选择在网元侧和网管侧查询历史性能，得到的数据不同，网管侧读取为错误。部分网元无历史性能。。请给出类似案例以供参考。


num_beams=2
. 查看UNM2000网管中，某些网元在网元侧和网管侧查询历史性能时，得到的答案不一样。在网管侧读取时，存在错误。而某些网元则没有历史性能。
2. 登录UNM2000网管后，在网元侧查询历史性能时，某些答案与网管侧不同。没有得到一致的信息。
3. 在UNM2000网管中，部分网元在查询历史性能时，仅在网管侧读取得到答案。而在网元侧，由于某些原因，无法读取到性能历史。
4. 假设在UNM2000网管中，两个网元分别位于不同的物理位置，一个网元在网元侧查询历史性能，另一个网元则在意念单元格内查询。结果表明，这两个网元得到的历史性能数据不同。
5. 登录UNM2000网管后，在网元侧查询历史性能，得到了一个错误答案。通过在网管侧重新阅读，发现读取历史性能的部分网元不在网管侧，这些网元在网管侧查询时，应该得到正确的历史性能数据。
6. 某些网元在UNM2000网管中，同时拥有历史性能在网元侧和网管侧。然而，某些情况下，这两个部分的数据并不一致。
ChatGLM2-6B：00网管中历史性能，登录后仅仅在网管侧查询了部分网元，然而部分网元在网管侧查询时，仍然得到了错误的历史性能数据。
 1. 查看UNM2000网管中，某些网元在网元侧和网管侧查询历史性能时，得到的答案不一样。在网管侧读取时，存在错误。而某些网元则没有历史性能。
2. 登录UNM2000网管后，在网元侧查询历史性能时，某些答案与网管侧不同。没有得到一致的信息。
3. 在UNM2000网管中，部分网元在查询历史性能时，仅在网管侧读取得到答案。而在网元侧，由于某些原因，无法读取到性能历史。
4. 假设在UNM2000网管中，两个网元分别位于不同的物理位置，一个网元在网元侧查询历史性能，另一个网元则在意念单元格内查询。结果表明，这两个网元得到的历史性能数据不同。
5. 登录UNM2000网管后，在网元侧查询历史性能，得到了一个错误答案。通过在网管侧重新阅读，发现读取历史性能的部分网元不在网管侧，这些网元在网管侧查询时，应该得到正确的历史性能数据。
6. 某些网元在UNM2000网管中，同时拥有历史性能在网元侧和网管侧。然而，某些情况下，这两个部分的数据并不一致。
7. 为了查询UNM2000网管中历史性能，登录后仅仅在网管侧查询了部分网元，然而部分网元在网管侧查询时，仍然得到了错误的历史性能数据

 #QUIT


In [12]:
#del model

In [ ]:
print("用户：你好，智能小助手\n")
response, history = model.chat(tokenizer, "你好，智能小助手", history=[],top_p= 0.7,
                                  temperature=0.75)
print("ChatGLM2-6B：\n",response)
print("\n------------------------------------------------\n用户：")

line = input()
while line:
    if line.strip()=="#QUIT": # 输入#QUIT退出对话循环
        break
    #inst = "作为烽火SSE助手，请帮忙回答如下问题：" #计算两个矩阵的乘积："
    #line = PROMPT_TEMPLATE.format_map({'instruction': line})
    if line.strip()=="我有一个新问题" or line.strip()=="#CLEAR":
        history=[]
    #line = inst  + line
    print(f"【formatted line】={line}")
    response, history = model.chat(tokenizer, line, history=history,top_p= 0.9,
                                  temperature=0.7,num_beams=1) # 量化后 num_beams只能=1 否则报错
    print("ChatGLM2-6B：\n", response.replace("\\n",'\n'))
    print("\n------------------------------------------------\n用户：")
    line = input()

In [ ]:
!pip install docx2txt

In [ ]:
!pip install pypandoc

In [ ]:
import glob
import pypandoc

data_path = "/kaggle/working/docdata"
docxFilename = '/kaggle/working/docdata/1.docx'
output = pypandoc.convert_file(docxFilename, 'plain', outputfile="somefile.txt")
assert output == ""




directory = glob.glob(f'{data_path}/**/*.doc', recursive=True)

# for file_name in directory:

#     with open(file_name, 'rb') as infile:
#         outfile = open(file_name[:-4]+'.txt', 'w', encoding='utf-8')
#         doc = docx2txt.process(infile)

#         outfile.write(doc)

#         outfile.close()
#     #infile.close()

# print("=========")
# print("All done!")

In [ ]:
#mv /kaggle/working/docdata/1.doc /kaggle/working/docdata/1.docx

In [ ]:
#cd /kaggle/working/docdata

In [ ]:
# 没有rename 命令
#!rename 's/\.doc/\.docx/' *